In [50]:
import os
from dotenv import load_dotenv

load_dotenv()
MODEL = "mistral"

In [51]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)
print(model.invoke("Tell me a joke"))

 Why don't scientists trust atoms?

Because they make up everything, but some of them are pretty neutron-tactful!


In [38]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser
print(chain.invoke("Tell me a joke"))

 Why did the chicken cross the road?
To get to the other side!


In [26]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/isca22.pdf")
pages  = loader.load_and_split()
pages

[Document(metadata={'source': 'data/isca22.pdf', 'page': 0}, page_content='Increasing Ising Machine Capacity with Multi-Chip\nArchitectures\nAnshujit Sharma, Richard Afoakwa, Zeljko Ignjatovic and Michael Huang\n{anshujitsharma,richard.afoakwa,zeljko.ignjatovic,michael.huang}@rochester.edu\nDepartment of Electrical and Computer Engineering, University of Rochester\nRochester, New York, USA\nABSTRACT\nNature has inspired a lot of problem solving techniques over the\ndecades. More recently, researchers have increasingly turned to\nharnessing nature to solve problems directly. Ising machines are a\ngood example and there are numerous research prototypes as well\nas many design concepts. They can map a family of NP-complete\nproblems and derive competitive solutions at speeds much greater\nthan conventional algorithms and in some cases, at a fraction of\nthe energy cost of a von Neumann computer.\nHowever, physical Ising machines are often fixed in its problem\nsolving capacity. Without an

In [39]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know"

Context: {context}
Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't
answer the question, reply "I don't know"

Context: Here is some context
Question: Here is a question



In [40]:
chain = prompt | model | parser
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question']}

In [41]:
chain.invoke(
    {
        "context" : "The name I was given was Anshujit",
        "question": "What is my name?"
    }
)

' Anshujit'

In [42]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [43]:
# retriever can be from google search for example
retriever = vectorstore.as_retriever()

# return the 4 topmost relevant documents for the given concept
retriever.invoke("Ising machine")

[Document(metadata={'source': 'data/isca22.pdf', 'page': 13}, page_content='00546-4\n[50] Hongyi Wang, Scott Sievert, Zachary Charles, Shengchao Liu, Stephen Wright,\nand Dimitris Papailiopoulos. 2018. ATOMO: Communication-Efficient Learning\nvia Atomic Sparsification. In Proceedings of the 32nd International Conference\non Neural Information Processing Systems (Montréal, Canada) (NIPS’18) . Curran\nAssociates Inc., Red Hook, NY, USA, 9872–9883.\n[51] Tianshi Wang and Jaijeet Roychowdhury. 2019. OIM: Oscillator-Based\nIsing Machines for Solving Combinatorial Optimisation Problems.\narXiv:1903.07163 [cs.ET]\n[52] Tianshi Wang, Leon Wu, and Jaijeet Roychowdhury. 2019. New Computational\nResults and Hardware Prototypes for Oscillator-Based Ising Machines. In Pro-\nceedings of the 56th Annual Design Automation Conference 2019 (Las Vegas, NV,\nUSA) (DAC ’19) . Association for Computing Machinery, New York, NY, USA,\nArticle 239, 2 pages. https://doi.org/10.1145/3316781.3322473\n[53] Jianqia

In [45]:
from operator import itemgetter

chain = (
    {
        "context" : itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

print(chain.invoke({"question" : "What are Ising machines?"}))


Ising machines use magnetic fields and spins on atoms that can either align with or against the field.


In [47]:
questions = [
    "How does Ising machine solve QUBO problem?",
    "What is the motivation of the paper?"
]

for question in questions:
    print(f"Question: {question}")
    answer = chain.invoke({"question" : question})
    print(f"Answer: {answer}")
    print()

Question: How does Ising machine solve QUBO problem?
Answer: 

Question: What is the motivation of the paper?
Answer: The authors discuss how different types of solvers work from first principles and explore the high-dimensional energy landscape sufficiently to achieve a good solution. They use simulated annealing (SA) and BRIM as examples, both exploring 148K and 115K different states respectively to arrive at comparable solution quality.



In [49]:
for s in chain.stream({"question" : "What is the problem that the paper is trying to solve?"}):
    print(s, end="", flush=True)

The problem the paper aims to solve can be described as optimizing a system represented by an objective function, where some variables have to satisfy certain constraints.

In [21]:
chain.batch([{"question" : q} for q in questions])

['Previous Ising machine implementations have faced several challenges and limitations, including:\n\n1. Memory constraints: Many previous Ising machine implementations were limited by the available memory on a single chip or device, which hindered their ability to solve large-scale optimization problems.\n2. Computational complexity: The computational complexity of traditional Ising machine algorithms can be high, particularly for large problem sizes, which can limit their scalability.\n3. Communication overhead: In distributed computing settings, the communication overhead between processors can be significant, especially when using traditional parallelization strategies.\n4. Limited parallelism: Traditional Ising machine implementations often have limited parallelism, which can result in slower convergence rates and reduced scalability.\n5. Non-optimal problem solving: Some previous Ising machine implementations may not always find the global optimum for a given problem, which can l